In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine


# Conexão com o banco de dados 
connection_string = "mysql://{user}:{passw}@{host}/{db}".format(user='admin',
                                                                passw='0WHPq8U2yJ9JgHVjsRBM',
                                                                host='interview-2.ck1h5ksgzpiq.us-east-1.rds.amazonaws.com',
                                                                db='innodb')

mysql_engine = create_engine(connection_string)

In [2]:
%%time

# extração do dados
query = '''

select * from sales_data

'''

df = pd.read_sql(query, con = mysql_engine)
df_copy = df.copy()

Wall time: 10min 54s


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599829 entries, 0 to 1599828
Data columns (total 14 columns):
purchase_id                        1599829 non-null int64
product_id                         1599828 non-null float64
affiliate_id                       1599828 non-null float64
producer_id                        1599828 non-null float64
buyer_id                           1599828 non-null float64
purchase_date                      1599828 non-null datetime64[ns]
product_creation_date              1599828 non-null datetime64[ns]
product_category                   1599828 non-null object
product_niche                      1599828 non-null object
purchase_value                     1599828 non-null float64
affiliate_commission_percentual    1599828 non-null float64
purchase_device                    1599828 non-null object
purchase_origin                    1599828 non-null object
is_origin_page_social_network      1599829 non-null object
dtypes: datetime64[ns](2), float64(6), i

In [4]:
# verifica valores Nan
for column in df:
    if df[column].isnull().any():
       print('{0} has {1} null values'.format(column, df[column].isnull().sum()))

product_id has 1 null values
affiliate_id has 1 null values
producer_id has 1 null values
buyer_id has 1 null values
purchase_date has 1 null values
product_creation_date has 1 null values
product_category has 1 null values
product_niche has 1 null values
purchase_value has 1 null values
affiliate_commission_percentual has 1 null values
purchase_device has 1 null values
purchase_origin has 1 null values


In [5]:
# remove valores Nan
df = df.dropna()

In [6]:
# tipos de dados das variáveis
df.dtypes

purchase_id                                 int64
product_id                                float64
affiliate_id                              float64
producer_id                               float64
buyer_id                                  float64
purchase_date                      datetime64[ns]
product_creation_date              datetime64[ns]
product_category                           object
product_niche                              object
purchase_value                            float64
affiliate_commission_percentual           float64
purchase_device                            object
purchase_origin                            object
is_origin_page_social_network              object
dtype: object

In [7]:
# Ajustes dos tipos de dados para o formato adequado

ints = ['product_id', 'affiliate_id', 'producer_id', 'buyer_id', 'affiliate_commission_percentual']

for col in ints:
    df[col] = df[col].astype(int)
    
objects = ['purchase_id',  'product_id', 'affiliate_id', 'producer_id', 'buyer_id', 'product_category',
           'product_niche', 'purchase_device', 'purchase_origin', 'is_origin_page_social_network']

for col in objects:
    df[col] = df[col].astype(str)

In [8]:
df.dtypes

purchase_id                                object
product_id                                 object
affiliate_id                               object
producer_id                                object
buyer_id                                   object
purchase_date                      datetime64[ns]
product_creation_date              datetime64[ns]
product_category                           object
product_niche                              object
purchase_value                            float64
affiliate_commission_percentual             int32
purchase_device                            object
purchase_origin                            object
is_origin_page_social_network              object
dtype: object

In [9]:
# Estatísticas dos dados numéricos
round(df.describe())

,purchase_value,affiliate_commission_percentual
count,1599828.0,1599828.0
mean,0.0,8.0
std,1.0,18.0
min,-1.0,0.0
25%,-0.0,0.0
50%,-0.0,0.0
75%,0.0,0.0
max,125.0,100.0


In [10]:
# Para não trabalhar com valores negativos, transformei os valores das vendas em valores "reais" novamente, para isso chutei uma média e um desvio padrão.
# Note que nehuma análise foi feita para estimar os valores de desvio padrão ou média, apenas chutei valores para transformar os dados e facilitar a compreensão.

def reverse_zscore(pandas_series, mean, std):
    original = pandas_series * std + mean
    return original

df['purchase_value'] = round(reverse_zscore(df['purchase_value'], 300, 450), 2)

In [11]:
df.affiliate_commission_percentual = df.affiliate_commission_percentual / 100

In [12]:
df.head()

,purchase_id,product_id,affiliate_id,producer_id,buyer_id,purchase_date,product_creation_date,product_category,product_niche,purchase_value,affiliate_commission_percentual,purchase_device,purchase_origin,is_origin_page_social_network
1,1663958,6640,209372,116238,1200397,2016-06-26 12:00:00,2011-03-19 15:47:36,Video,Presentation skills,180.61,0.5,Smart TV,Origin ef2b,"0,0"
2,1677087,2350,141418,2821,1083764,2016-06-26 12:00:00,2010-07-05 01:50:15,Podcast,Child psychology,220.32,0.6,Smart TV,Origin ef2b,"0,0"
3,2017360,35669,618642,618642,1436106,2016-06-26 12:00:00,2012-06-13 02:59:37,Podcast,Presentation skills,88.95,0.0,Smart TV,Origin ef2b,"0,0"
4,2017379,57998,1164511,70388,1436118,2016-06-26 12:00:00,2013-05-07 08:51:31,Podcast,Anxiety management,119.47,0.5,Smart TV,Origin ef2b,"0,0"
5,2017382,58329,1261488,221253,1386357,2016-06-26 12:00:00,2013-05-12 08:12:06,Podcast,Teaching English,96.38,0.5,Smart TV,Origin ef2b,"0,0"


In [13]:
round(df.describe(),2)

,purchase_value,affiliate_commission_percentual
count,1599828.00,1599828.00
mean,300.00,0.08
std,450.00,0.18
min,56.44,0.00
25%,96.14,0.00
50%,142.50,0.00
75%,329.22,0.00
max,56352.45,1.00


In [14]:
df.to_csv('data_source_hotmart.csv', header=True, index=False, encoding='UTF-8', decimal=',')